**We download stock market data from the following website.**

Source: https://query1.finance.yahoo.com

In [1]:
import sys
import requests

import pandas as pd

from tqdm import tqdm_notebook as tqdm
from concurrent.futures import ThreadPoolExecutor

# Helpers

In [2]:
BASEURL = 'https://query1.finance.yahoo.com'

In [3]:
notnone = lambda x: x is not None

# Datasets

In [4]:
stocks = pd.read_csv('../datasets/stocks.csv.gz', index_col=0)

stocks.head()

,ticker,name,exchange,category name,country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA


In [5]:
stocks.describe(include='all')

,ticker,name,exchange,category name,country
count,106328,96676,95092,20728,95091
unique,106328,64859,67,214,41
top,BWI.SG,FINANC/TERMO,PNK,Industrial Metals & Minerals,USA
freq,1,439,14602,1292,22169


In [6]:
tickers = stocks['ticker']

tickers.head()

0    OEDV
1    AAPL
2     BAC
3    AMZN
4       T
Name: ticker, dtype: object

# Data infos

In [7]:
def download_info(ticker):
    """Download meta information from the given ticker."""
    url = f"{BASEURL}/v7/finance/quote?symbols={ticker}"
    
    try:
        res = requests.get(url=url).json()
        result = res['quoteResponse']['result'][0]
        
        return result
    except Exception as e:
        # fail silently
        return None

In [8]:
with ThreadPoolExecutor() as executor:
    gen = executor.map(download_info, tickers)
    infos = pd.DataFrame(filter(notnone, tqdm(gen, total=len(tickers))))

print(infos.shape)

infos.head()


(82108, 75)


,language,region,quoteType,quoteSourceName,exchangeDataDelayedBy,fullExchangeName,esgPopulated,tradeable,triggerable,regularMarketTime,...,forwardPE,ytdReturn,trailingThreeMonthReturns,trailingThreeMonthNavReturns,openInterest,underlyingSymbol,contractSymbol,expireDate,expireIsoDate,headSymbolAsString
0,en-US,US,MUTUALFUND,Delayed Quote,0.0,YHD,False,False,False,1.561760e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en-US,US,EQUITY,Nasdaq Real Time Price,0.0,NasdaqGS,False,True,True,1.567016e+09,...,15.996875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,en-US,US,EQUITY,Nasdaq Real Time Price,0.0,NYSE,False,True,True,1.567016e+09,...,8.879934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,en-US,US,EQUITY,Nasdaq Real Time Price,0.0,NasdaqGS,False,True,True,1.567016e+09,...,53.036423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,en-US,US,EQUITY,Nasdaq Real Time Price,0.0,NYSE,False,True,True,1.567016e+09,...,9.624310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
infos.describe(include='all')

,language,region,quoteType,quoteSourceName,exchangeDataDelayedBy,fullExchangeName,esgPopulated,tradeable,triggerable,regularMarketTime,...,forwardPE,ytdReturn,trailingThreeMonthReturns,trailingThreeMonthNavReturns,openInterest,underlyingSymbol,contractSymbol,expireDate,expireIsoDate,headSymbolAsString
count,82108,82108,82108,37384,78098.000000,78098,82108,82108,82108,7.713500e+04,...,1.105800e+04,159.000000,161.000000,110.000000,550.0,461,529,8.700000e+01,87,17
unique,1,1,11,2,NaN,68,1,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,461,2,NaN,15,17
top,en-US,US,EQUITY,Delayed Quote,NaN,YHD,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,HDFCSENETF.NS,True,NaN,2029-12-31T00:00:00Z,ZWTKRB-A
freq,82108,82108,68145,33164,NaN,11424,82108,68415,77472,NaN,...,NaN,NaN,NaN,NaN,NaN,1,512,NaN,62,1
mean,NaN,NaN,NaN,NaN,1.716433,NaN,NaN,NaN,NaN,1.547038e+09,...,4.012157e+04,9.962453,1.242112,0.558909,0.0,NaN,NaN,1.799777e+09,NaN,NaN
std,NaN,NaN,NaN,NaN,5.226918,NaN,NaN,NaN,NaN,1.678285e+08,...,6.404696e+06,13.732888,6.146069,5.171344,0.0,NaN,NaN,1.487471e+08,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000e+00,...,-3.862825e+08,-22.380000,-20.600000,-20.730000,0.0,NaN,NaN,1.473293e+09,NaN,NaN
25%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,1.565330e+09,...,5.033942e+00,0.710000,-1.440000,-1.575000,0.0,NaN,NaN,1.573560e+09,NaN,NaN
50%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,1.566973e+09,...,1.092902e+01,8.160000,1.320000,0.960000,0.0,NaN,NaN,1.893370e+09,NaN,NaN
75%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,1.567005e+09,...,1.909479e+01,18.440000,4.120000,3.077500,0.0,NaN,NaN,1.893370e+09,NaN,NaN


# Data extras

In [10]:
extras = pd.merge(stocks, infos, how='inner', left_on='ticker', right_on='symbol')

extras.head()

,ticker,name,exchange_x,category name,country,language,region,quoteType,quoteSourceName,exchangeDataDelayedBy,...,forwardPE,ytdReturn,trailingThreeMonthReturns,trailingThreeMonthNavReturns,openInterest,underlyingSymbol,contractSymbol,expireDate,expireIsoDate,headSymbolAsString
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA,en-US,US,MUTUALFUND,Delayed Quote,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA,en-US,US,EQUITY,Nasdaq Real Time Price,0.0,...,15.996875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA,en-US,US,EQUITY,Nasdaq Real Time Price,0.0,...,8.879934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA,en-US,US,EQUITY,Nasdaq Real Time Price,0.0,...,53.036423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA,en-US,US,EQUITY,Nasdaq Real Time Price,0.0,...,9.624310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
extras.describe(include='all')

,ticker,name,exchange_x,category name,country,language,region,quoteType,quoteSourceName,exchangeDataDelayedBy,...,forwardPE,ytdReturn,trailingThreeMonthReturns,trailingThreeMonthNavReturns,openInterest,underlyingSymbol,contractSymbol,expireDate,expireIsoDate,headSymbolAsString
count,82099,80216,79005,20570,79005,82099,82099,82099,37375,78089.000000,...,1.105800e+04,159.000000,161.000000,110.000000,550.0,461,529,8.700000e+01,87,17
unique,82073,54594,64,214,41,1,1,11,2,NaN,...,NaN,NaN,NaN,NaN,NaN,461,2,NaN,15,17
top,M,ICICI Prudential Mutual Fund,PNK,Industrial Metals & Minerals,USA,en-US,US,EQUITY,Delayed Quote,NaN,...,NaN,NaN,NaN,NaN,NaN,HDFCSENETF.NS,True,NaN,2029-12-31T00:00:00Z,ZWTKRB-A
freq,9,34,11285,1281,18020,82099,82099,68145,33155,NaN,...,NaN,NaN,NaN,NaN,NaN,1,512,NaN,62,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.716631,...,4.012157e+04,9.962453,1.242112,0.558909,0.0,NaN,NaN,1.799777e+09,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.227186,...,6.404696e+06,13.732888,6.146069,5.171344,0.0,NaN,NaN,1.487471e+08,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,-3.862825e+08,-22.380000,-20.600000,-20.730000,0.0,NaN,NaN,1.473293e+09,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,5.033942e+00,0.710000,-1.440000,-1.575000,0.0,NaN,NaN,1.573560e+09,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,1.092902e+01,8.160000,1.320000,0.960000,0.0,NaN,NaN,1.893370e+09,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,1.909479e+01,18.440000,4.120000,3.077500,0.0,NaN,NaN,1.893370e+09,NaN,NaN


In [12]:
extras.to_csv('../datasets/extras.csv.gz')

# Data points

In [13]:
def download_data(ticker, range='5y', interval='1d', events='div,splits'):
    """Download market information for the given ticker and time range."""
    url = f"{BASEURL}/v8/finance/chart/{ticker}"
    
    params = dict(
        range=range,
        events=events,
        interval=interval,
    )
    
    try:
        res = requests.get(url=url, params=params).json()
        result = res['chart']['result'][0]
        splits = result['events']['splits']
        dividends = result['events']['dividends']
        indicators = result['indicators']['quote'][0]
                
        splits = pd.DataFrame(splits.values()).assign(ticker=ticker)
        dividends = pd.DataFrame(dividends.values()).assign(ticker=ticker)
        quotes = pd.DataFrame(indicators).assign(date=result['timestamp'], ticker=ticker)
        
        return quotes, splits, dividends
    except Exception as e:
        # fail silently
        return None

In [14]:
with ThreadPoolExecutor() as executor:
    gen = executor.map(download_data, tickers)
    values = filter(notnone, tqdm(gen, total=len(tickers)))
    quotes, splits, dividends = zip(*values)

allsplits = pd.concat(splits, sort=False)
allquotes = pd.concat(quotes, sort=False)
alldividends = pd.concat(dividends, sort=False)

allquotes.shape, allsplits.shape, alldividends.shape

((5816659, 7), (7086, 5), (38819, 3))

## Quotes

In [15]:
allquotes.head()

,high,close,open,low,volume,date,ticker
0,40.082039,39.817711,39.937859,39.697559,2778000.0,1409232600,AA
1,40.106071,39.913830,39.961891,39.577412,3811300.0,1409319000,AA
2,40.178162,40.058010,39.985920,39.793678,4474400.0,1409664600,AA
3,40.851002,40.682789,40.082039,40.058010,4523400.0,1409751000,AA
4,41.716080,41.091301,40.802940,40.634731,6497800.0,1409837400,AA


In [16]:
allquotes.describe(include='all')

,high,close,open,low,volume,date,ticker
count,5.777200e+06,5.777200e+06,5.777200e+06,5.777200e+06,5.777200e+06,5.816659e+06,5816659
unique,NaN,NaN,NaN,NaN,NaN,NaN,4969
top,NaN,NaN,NaN,NaN,NaN,NaN,ISFIN.IS
freq,NaN,NaN,NaN,NaN,NaN,NaN,1305
mean,1.516621e+03,1.476032e+03,1.483932e+03,1.442154e+03,3.919745e+06,1.489245e+09,NaN
std,1.080886e+05,1.045192e+05,1.058057e+05,1.019228e+05,1.802459e+08,4.532720e+07,NaN
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.409198e+09,NaN
25%,6.666670e+00,6.600000e+00,6.600000e+00,6.530000e+00,0.000000e+00,1.450103e+09,NaN
50%,1.937372e+01,1.922000e+01,1.921000e+01,1.906000e+01,6.001000e+03,1.489712e+09,NaN
75%,4.804000e+01,4.765000e+01,4.765960e+01,4.725000e+01,3.750785e+05,1.528438e+09,NaN


In [17]:
allquotes.to_csv('../datasets/quotes.csv.gz')

## Splits

In [18]:
allsplits.head()

,date,numerator,denominator,splitRatio,ticker
0,1478007000,1000,801,801/1000,AA
1,1475760600,1,3,3/1,AA
0,1559568600,4725,10000,10000/4725,DD
1,1554211800,1487,1000,1000/1487,DD
0,1412256600,14,1,1/14,MT


In [19]:
allsplits.describe(include='all')

,date,numerator,denominator,splitRatio,ticker
count,7.086000e+03,7.086000e+03,7086.000000,7086,7086
unique,NaN,NaN,NaN,532,4969
top,NaN,NaN,NaN,1/2,UBFO
freq,NaN,NaN,NaN,1138,11
mean,1.484535e+09,1.034722e+03,319.618261,NaN,NaN
std,4.517537e+07,5.622322e+04,5508.402633,NaN,NaN
min,1.409206e+09,0.000000e+00,0.000000,NaN,NaN
25%,1.441692e+09,2.000000e+00,1.000000,NaN,NaN
50%,1.482786e+09,5.000000e+00,5.000000,NaN,NaN
75%,1.523961e+09,2.600000e+01,30.000000,NaN,NaN


In [20]:
allsplits.to_csv('../datasets/splits.csv.gz')

## Dividends

In [21]:
alldividends.head()

,amount,date,ticker
0,0.07209,1462368600,AA
1,0.09000,1478093400,AA
2,0.07209,1415197800,AA
3,0.07209,1446647400,AA
4,0.07209,1431005400,AA


In [22]:
alldividends.describe(include='all')

,amount,date,ticker
count,38819.000000,3.881900e+04,38819
unique,NaN,NaN,4969
top,NaN,NaN,ITUB
freq,NaN,NaN,80
mean,63.660139,1.490452e+09,NaN
std,5404.631248,4.562170e+07,NaN
min,0.000000,1.409206e+09,NaN
25%,0.098003,1.451313e+09,NaN
50%,0.267500,1.491880e+09,NaN
75%,1.330000,1.529977e+09,NaN


In [23]:
alldividends.to_csv('../datasets/dividends.csv.gz')